In [1]:
# Re-start Jupyter server after this
# %pip install -U anywidget jupyterlab-widgets lsdb plotly reproject

In [2]:
from single_object import make_figure

In [3]:
from lsst.daf.butler import Butler

repo = "/repo/main"
instrument = "LSSTComCam"
release = "w_2025_06"
collection = f"LSSTComCam/runs/DRP/DP1/{release}/DM-48810"

butler = Butler(repo, collections=collection)
butler

In [4]:
from pathlib import Path

hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats") / release
# list dir
print(list(map(str, hats_path.iterdir())))

['/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/object_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/diaObject_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/diaObject_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/diaSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/diaObject', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/object_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/object', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/diaForcedSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/forcedSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/diaObject_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/object_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_06/source']


In [5]:
make_figure(2132951402815696813, butler, hats_path)

/sdf/data/rubin/user/kostya/linccf/.venv/lib/python3.11/site-packages/lsdb/dask/join_catalog_data.py:332: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(
